# Plot graph examples

For both concept maps and co-occurrence graphs

In [ ]:
from notebook_prelude import *

In [ ]:
def get_graph_dot(g):
    return nx.nx_pydot.to_pydot(g)

def save_dot_graph(g: nx.Graph, filename: str):
    g = get_graph_dot(g)
    with open(filename, 'w') as f:
        f.write(g.to_string())

def draw_graph(graph, layout = nx.layout.circular_layout, with_frame = True, node_color='#000000', node_size=16, edge_color='#000000'):
    fig, ax = plt.subplots(figsize=(6, 6))
    pos = None
    if layout:
        pos = layout(graph)
    nx.draw_networkx(graph, pos=pos, ax = ax, node_size = node_size, with_labels = False, node_color = node_color, arrows=False, edge_color=edge_color)
    ax.set_frame_on(with_frame)
    #cleanup_axes(ax)
    ax.grid(False)
    ax.set_xticks([])
    ax.set_yticks([])
    fig.tight_layout()
    return fig, ax

def get_candidates(X, min_nodes = 10, max_nodes = 15):
    return [x for x in X if nx.number_of_nodes(x) in range(min_nodes, max_nodes)]

In [ ]:


# Tuples of: (dataset_name, graph_type, (X, Y))
# For cooccurrence graphs, it will hold a (random) choice for each window size
graph_datasets = []
for dataset in dataset_helper.get_dataset_names_with_concept_map(limit_datasets=['ng20']):
    print('{:30} start'.format(dataset))
    graph_cache_files = dataset_helper.get_all_cached_graph_datasets(dataset_name=dataset)
    gml_graph_cache = [x for x in graph_cache_files if 'concept' in x][0]
    coo_graph_caches = [x for x in graph_cache_files if 'cooc' in x]
    
    def get_window_size(graph_cache_file):
        return graph_cache_file.split('cooccurrence_')[1].split('_')[0]
    
    coo_graphs_by_window_size = collections.defaultdict(lambda: [])
    for cache_file in coo_graph_caches:
        coo_graphs_by_window_size[get_window_size(cache_file)].append(cache_file)
    
    X_cmap, Y_cmap = dataset_helper.get_dataset_cached(gml_graph_cache)
    X_cmap = graph_helper.get_graphs_only(X_cmap)
    graph_datasets.append((dataset, 'Concept Map', (get_candidates(X_cmap), Y_cmap)))
    for window_size, cached_files in sorted(coo_graphs_by_window_size.items(), key=lambda x: x[0]):
        # Take random element from the co-occurence graph datasets
        coo_graph_cache = np.random.choice(cached_files)
        print('\tRetrieving co-occurence graphs for window_size={} ({})'.format(window_size, coo_graph_cache))
        X, Y = dataset_helper.get_dataset_cached(coo_graph_cache)
        X = graph_helper.get_graphs_only(X)
        X = get_candidates(X)
        graph_datasets.append((dataset, 'Cooccurrence\n$w={}$'.format(window_size), (X, Y)))
    break

gc.collect()

In [ ]:
df = pd.DataFrame(graph_datasets, columns = ['dataset', 'graph_type', 'graph_dataset'])

In [ ]:
NUM_GRAPHS_PER_TYPE = 3

for dataset, data in df.groupby('dataset'):
    fig, axes = plt.subplots(ncols=data.graph_type.value_counts().size, nrows=NUM_GRAPHS_PER_TYPE)
    for idx, row_ax in enumerate(axes):
        print('Row: {}/{}'.format(idx + 1, len(axes)))
        for (_, item), ax in zip(data.iterrows(), row_ax):
            graph_type = item.graph_type
            if idx == 0:
                ax.set_title(graph_type)
            

            X, Y = item.graph_dataset
            candidates = [x for x in X if nx.number_of_nodes(x) in range(4, 14)]
            random_graph = np.random.choice(candidates)
            pos = None if not graph_type.count('Concept') else nx.layout.circular_layout(random_graph)
            nx.draw_networkx(random_graph, ax = ax, node_size = 16, with_labels = False, node_color = '#000000', arrows=False)
            #cleanup_axes(ax)
            ax.grid(False)
            ax.set_xticks([])
            ax.set_yticks([])
    fig.tight_layout(h_pad=2, w_pad=2)
    
    for ax in axes.flatten():
        (x_min, x_max), (y_min, y_max) = ax.get_xlim(), ax.get_ylim()
        #ax.set_xlim((0, 1))
        #ax.axvline(x_min)
        #ax.axvline(x_max)
    plt.show()
    save_fig(fig, 'examples_graphs')

## Presentation: Concept map example, big

In [ ]:
X, Y = dataset_helper.get_concept_map_for_dataset('ng20')
X = graph_helper.get_graphs_only(X)

In [ ]:
max_nodes = max([nx.number_of_nodes(x) for x in X])
print(max_nodes)
TUD_RED = '#9c1e23'
BLACK = '#000000'

node_color = BLACK
edge_color = '#555555'
node_size = 20
node_range = (300, 500)

X_ = get_candidates(X, min_nodes=node_range[0], max_nodes=node_range[1])
assert len(X_)
idx = np.random.choice(len(X_))
X_[idx].nodes()
random_graph = X_[idx]
for pos in [None, nx.layout.circular_layout, nx.layout.fruchterman_reingold_layout, nx.layout.shell_layout, nx.layout.spectral_layout, nx.layout.spring_layout, nx.layout.random_layout]:
    layout_name = pos.__name__ if pos else 'normal'
    fig, ax = draw_graph(random_graph, layout=pos, with_frame=False, node_color=node_color, node_size=node_size, edge_color=edge_color)
    save_fig(fig, 'example_concept_map_{}'.format(layout_name), folder='tmp/examples_concept_map')
    plt.show()
#np.random.choice(X_)

## Presentation: Concept map example

In [ ]:
g = nx.DiGraph()
g.add_edge('cigarette smoking', 'breath', dict(label='affects'))
g.add_edge('drinking', 'breath', dict(label='affects'))
g.add_edge('breath', 'important', dict(label='is'))

nx.draw_networkx(g)
g_pydot = get_graph_dot(g)

FILE = 'tmp/example_presentation_concept_map.dot'
with open(FILE, 'w') as f:
    f.write(g_pydot.to_string())

## Presentation: Co-occurrence

In [ ]:
sentence = r'Universal history is the history of a few metaphors.'

from utils import cooccurrence
word2idx, idx2word, mat = cooccurrence.get_coocurrence_matrix(sentence.lower(), window_size=1)
graph = graph_helper.convert_from_numpy_to_nx(word2idx, idx2word, mat)
nx.draw_networkx(graph)

## Presentation: Simple Triangle/Rectangle Graph kernel

In [ ]:
#layout = nx.layout.
layout = None

g = nx.Graph()
g.add_edge('A', 'B')
g.add_edge('A', 'C')
g.add_edge('B', 'C')



g_rect = g.copy()
g_rect.add_edge('A', 'E')
g_rect.add_edge('B', 'E')

save_dot_graph(g_rect, 'tmp/graph_examples/graph_rectangle.dot')
save_dot_graph(g, 'tmp/graph_examples/graph_triangle.dot')

draw_graph(g, layout=layout)
draw_graph(g_rect, layout=layout)


In [ ]:
!fdp -Tpng -otmp/graph_examples/graph_triangle.png tmp/graph_examples/graph_triangle.dot
!circo -Tpng -otmp/graph_examples/graph_rectangle.png tmp/graph_examples/graph_rectangle.dot